In [ ]:
import glob
import numpy as np
import os
import tqdm

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
fig = plt.figure()
matplotlib.style.use( '/Users/zhafen/repos/clean-bold/clean-bold.mplstyle' )

In [ ]:
import anvil

In [ ]:
CHUNK_SIZE = 16
CHUNKS_PER_REGION = 32
REGION_SIZE = CHUNK_SIZE * CHUNKS_PER_REGION

# Parameters

In [ ]:
world_dir = '/Users/zhafen/data/minecraft/saves/aternos/world'

In [ ]:
search_block_id = 'redstone_block'

In [ ]:
regions_to_check = [
    ( -1, 1 ),
    ( -2, 1 ),
]

# Load Data

In [ ]:
if regions_to_check is None:
    region_fps = glob.glob( os.path.join( world_dir, 'region', '*' ) )
else:
    region_fps = [ os.path.join( world_dir, 'region', 'r.{}.{}.mca'.format( *_ ) ) for _ in regions_to_check ]

In [ ]:
# Get the possible y values
region_random = anvil.Region.from_file( region_fps[0] )
chunk_random = anvil.Chunk.from_region( region_random, 0, 0 )
section_height_range = anvil.chunk._section_height_range( chunk_random.version )
y_range = np.arange( section_height_range.start * CHUNK_SIZE, section_height_range.stop * CHUNK_SIZE )

In [ ]:
# Convert indices to xs, ys, and zs, using index = y * 16**2 + z * 16 + x
inds = np.arange( 16**2 * y_range.size )

y_inds = inds // 16**2
z_inds = ( inds - y_inds * 16**2 ) // 16
x_inds = ( inds - y_inds * 16**2 - z_inds * 16 )

In [ ]:
# Loop through and get data for all regions
regions_containing_block = []
for i, region_fp in enumerate( region_fps ):
    region = anvil.Region.from_file( region_fp )
    
    # Get region coords
    _, x_region, z_region, _ = os.path.basename( region_fp ).split( '.' )
    x_region, z_region = int( x_region ), int( z_region )
    
    print( '{} / {}: searching region ({}, {})'.format( i+1, len( region_fps ), x_region, z_region ) )

    # Extract data in string format
    for i_chunk in tqdm.tqdm( range( CHUNKS_PER_REGION ) ):
        for k_chunk in range( CHUNKS_PER_REGION ):

                # You can also provide the region file name instead of the object
                try:
                    chunk = anvil.Chunk.from_region( region, i_chunk, k_chunk )
                except anvil.chunk.ChunkNotFound as e:
                    continue

                for index, block in enumerate( chunk.stream_chunk() ):
                    
                    x = x_region * REGION_SIZE + i_chunk * CHUNK_SIZE + x_inds[index]
                    z = z_region * REGION_SIZE + k_chunk * CHUNK_SIZE + z_inds[index]
                    y = y_inds[index]

                    if block.id == search_block_id:
                        print( 'Found block at ({}, {}, {})'.format( x, y, z ) )
                        regions_containing_block.append( region_fp )

In [ ]:
-1, 1
-2, 1

In [ ]:
-1, 0